<h2>Q-Learning vs SARSA</h2>
This is a python implementation that shows the difference between Q-Learning and SARSA. It is designed as an easy way to highlight the key difference between the two algorithms in order to make the methods more accessible. 

In [1]:
import numpy as np
import grid_

<h2>Hyperparameters</h2>

In [2]:
GAMMA = 0.95
ALPHA = 0.1
STEP_COST = -0.05
ALL_ACTIONS = ('U', 'D', 'L', 'R')

<h2>Functions</h2>
<p>set_q:  Initializes action values to zero and also sets state/action counts<\p>
<p>best_action: Chooses the best action foor a given state</p>
<p>random_action: Returns a random action based off exploration vs. exploitation tradeoff. Exploration will be time decayed.</p> 

In [3]:
def set_q(grid):
    Q = {}
    counts_sa = {}
    states = grid.all_states
    for s in states:
        Q[s] = {}
        counts_sa[s] = {}
        for a in ALL_ACTIONS:
            Q[s][a] = 0
            counts_sa[s][a] = 1.0
    return Q, counts_sa

def best_action(choices):
    max_action = None
    max_val = -10000

    for k, v in choices.items():
        if v > max_val:
            max_val = v
            max_action = k
    return max_action, max_val

def random_action(action, e):
    p = np.random.random()
    if p < (1 - e):
        return action
    else:
        return np.random.choice(ALL_ACTIONS)

<h2>Important Assumption</h2>
<p>The two methods will make the same action at every step. It is possible that the "max action" and thus the actions could diverge because the two methods could update differently at the beginning. However, since each action is a random action we are free to make this assumption.</p>

In [6]:
def game():
    state = (1, 0)
    grid = grid_.Grid(state)
    Q_learn, action_counts = set_q(grid)
    Q_sarsa, _ = set_q(grid)

    print('REWARDS')
    grid.print_rewards()
    t = 1.0
    for n in range(10001):
        if n % 100 == 0:
            t += 0.005

        action, val = best_action(Q_learn[state])
        action = random_action(action, 0.5/t)
        while not grid.game_over():
            new_state = grid.move(action) 
            reward = grid.rewards.get(new_state, 0) + STEP_COST
            alpha = ALPHA / action_counts[state][action]
            action_counts[state][action] += 0.005
            q_val = Q_learn[state][action]
            max_action, max_st_val = best_action(Q_learn[new_state])
            next_action = random_action(max_action, 0.5/t)
            next_st_val = Q_sarsa[new_state][next_action]

            #KEY DIFFERENCE:  
            #Q-learning uses the max value to update the action value
            #SARSA uses the value of the next state:
            Q_learn[state][action] = Q_learn[state][action] + alpha*(reward + GAMMA * max_st_val - Q_learn[state][action])
            Q_sarsa[state][action] = Q_sarsa[state][action] + alpha*(reward + GAMMA * next_st_val - Q_sarsa[state][action])

            action = next_action
            state = new_state

        state = (1, 0)
        grid = grid_.Grid(state)


        if n % 500 == 0:
            print('ITERATION: ' + str(n) + '\n')
            print('Q Learning')
            grid.state_vals = grid.update_state_vals(Q_learn, default=False)
            grid.print_grid()

            print('SARSA')
            grid.state_vals = grid.update_state_vals(Q_sarsa, default=False)
            grid.print_grid()
            print('-'*50)

In [7]:
game()


REWARDS
---------------------
|    |    |    |    |
---------------------
|    |    |    |    |
---------------------
|    |    |    | -1 |
---------------------
|    |    |    | 1  |
---------------------


ITERATION: 0

Q Learning
-------------------------------------
|  0.0   |  0     |  0     |  0     |
-------------------------------------
|  -0.01 |  0.0   |    X   |  0     |
-------------------------------------
|  -0.01 |   X    |  0     |  0     |
-------------------------------------
|  -0.01 |  0.0   |  0.1   |  0     |
-------------------------------------


SARSA
-------------------------------------
|  0.0   |  0     |  0     |  0     |
-------------------------------------
|  -0.01 |  0.0   |    X   |  0     |
-------------------------------------
|  -0.01 |   X    |  0     |  0     |
-------------------------------------
|  -0.01 |  0.0   |  0.1   |  0     |
-------------------------------------


--------------------------------------------------
ITERATION: 500

Q Lear